In [1]:
import pandas as pd
import datetime as dt
from datetime import date
from pathlib import Path

In [2]:
today = date.today()
today_str = today.strftime("%Y%m%d")
print(today_str)


20190508


In [3]:
if today<(dt.date(today.year, 9, 1)):
    acad_yr = today.year
else:
    acad_yr = today.year - 1

print(f"acad_yr={acad_yr:d}")

acad_yr=2019


In [4]:
data_path = Path(r"C:\JW\Python\Admissions\funnel")
data_store = data_path / "data/stage_data"

df = pd.read_hdf(data_store, key="weekly")
df = df[(df["year_term"] > "2011.Spring")]


In [5]:
b_yr = 2012
e_yr = 2019
term = 'Fall'

terms = [f"{y}.{term}" for y in range(b_yr, e_yr+1)]
stages = [ 'Applied', 'Accepted', 'Deposited']


In [6]:
summ = df.groupby(["year_term", "stage"]).sum()
summ_t = summ.transpose()


In [7]:
print(summ.shape)
summ.head()


(48, 54)


00  01  02  03  04  05  06  07  08  09  ...   44   45  \
year_term   stage                                              ...             
2012.Fall   Accepted    0   1   1   1   1   1   1   1   2   2  ...  416  413   
            Applied     0   1   1   1   2   5   6   7  11  14  ...  585  583   
            Deposited   0   3   3   3   3   3   3   3   4   4  ...  368  368   
2012.Spring Accepted    0   0   0   0   0   0   0   0   0   0  ...    1    1   
            Applied     0   0   0   0   0   1   1   1   1   1  ...    2    2   

                        46   47   48   49   50   51   52   53  
year_term   stage                                              
2012.Fall   Accepted   396  395  359  360  343  290  276  276  
            Applied    562  557  509  510  493  436  423  423  
            Deposited  369  369  372  372  373  365  365  365  
2012.Spring Accepted     1    1    1    1    1    1    1    1  
            Applied      2    2    2    2    2    2    2    2  

[5 rows x 54 columns]

In [8]:
summ_t.head()


year_term 2012.Fall                   2012.Spring                   2013.Fall  \
stage      Accepted Applied Deposited    Accepted Applied Deposited  Accepted   
00                0       0         0           0       0         0         0   
01                1       1         3           0       0         0        18   
02                1       1         3           0       0         0        20   
03                1       1         3           0       0         0        23   
04                1       2         3           0       0         0        24   

year_term                   2013.Spring  ... 2018.Fall 2018.Spring          \
stage     Applied Deposited    Accepted  ... Deposited    Accepted Applied   
00              0         0           0  ...         0           0       0   
01             83         9           8  ...         7           2       4   
02             73         9           8  ...         6           2       4   
03             77        10           8  ...         6           2       5   
04             93        13          13  ...         6           2       6   

year_term           2019.Fall                   2019.Spring                    
stage     Deposited  Accepted Applied Deposited    Accepted Applied Deposited  
00                0         0       0         0           0       0         0  
01                3         2      19         1           2       8         2  
02                3         2      29         1           4      11         2  
03                3         2      37         1           5      13         3  
04                3         2      46         1           5      14         3  

[5 rows x 48 columns]

In [9]:
def adm_week(d):
    """
    returns calendar week number and Admissions Week Number for a given date, d
    """
    year = d.year
    if d >= date(year, 9, 1):
        adm_year_start = year
    else:
        adm_year_start = year - 1

    week_number = d.isocalendar()[1]
    adm_start_week = date(adm_year_start, 9, 1).isocalendar()[1]

    if week_number >= adm_start_week:
        adm_week_number = (week_number - adm_start_week) + 1
    else:
        adm_week_number = 53 + (week_number - adm_start_week)

    return (week_number, adm_week_number)


week_number, adm_week_number = adm_week(today)
print(week_number, adm_week_number)


19 37


In [10]:
def adm_week_end_date(awn, acad_yr):
    """
    returns calendar date for the Friday of the given Admissions Week Number, awn
        and academic year, acad_yr
    """
    
    adm_start_date = dt.date((acad_yr-1), 9, 1)
    adm_start_week = adm_start_date.isocalendar()[1]
    
    if awn <= (53 - adm_start_week):
        wn = adm_start_week + awn - 1
        year = acad_yr - 1
    else:
        wn = awn - (53 - adm_start_week) - 1
        year = acad_yr

    week_end = dt.datetime.strptime(f'{year}-{wn}-1', "%Y-%W-%w").date() + dt.timedelta(days=6.9)
    
    return(week_end)

print(today, adm_week_end_date(adm_week_number, acad_yr))

2019-05-08 2019-05-12


In [11]:
week = adm_week(today)[1]


In [12]:
stg = "Deposited"
for t in terms:
    print(t, str(week), summ.loc[(t, stg)][str(week)])

print(terms[-1], terms[-2], 
    summ.loc[(terms[-1], stg)][str(week)] - summ.loc[(terms[-2], stg)][str(week)]
)


2012.Fall 37 327
2013.Fall 37 270
2014.Fall 37 223
2015.Fall 37 263
2016.Fall 37 258
2017.Fall 37 198
2018.Fall 37 186
2019.Fall 37 176
2019.Fall 2018.Fall -10


In [13]:
day2 = date(acad_yr, 5, 1)
print(day2)
week2 = adm_week(day2)[1]
print(week2)

for t in reversed(terms):
    print(t, str(week2), summ.loc[(t, stg)][str(week2)])

for t in reversed(terms):
    print(summ.loc[(t, stg)][str(week2)])


2019-05-01
36
2019.Fall 36 174
2018.Fall 36 182
2017.Fall 36 191
2016.Fall 36 249
2015.Fall 36 254
2014.Fall 36 213
2013.Fall 36 268
2012.Fall 36 311
174
182
191
249
254
213
268
311


In [14]:
day2 = date(acad_yr, 8, 25)
print(day2)
week2 = adm_week(day2)[1]
print(week2)

for t in reversed(terms):
    print(t, str(week2), summ.loc[(t, stg)][str(week2)])

for t in reversed(terms):
    print(summ.loc[(t, stg)][str(week2)])


2019-08-25
52
2019.Fall 52 176
2018.Fall 52 245
2017.Fall 52 260
2016.Fall 52 286
2015.Fall 52 319
2014.Fall 52 304
2013.Fall 52 299
2012.Fall 52 365
176
245
260
286
319
304
299
365


In [15]:
print("Now to end of year")
for t in reversed(terms):
    print(t, str(week), summ.loc[(t, stg)][str(week)],
        "52", summ.loc[(t, stg)]["52"],
        summ.loc[(t, stg)]["52"] - summ.loc[(t, stg)][str(week)]
    )


Now to end of year
2019.Fall 37 176 52 176 0
2018.Fall 37 186 52 245 59
2017.Fall 37 198 52 260 62
2016.Fall 37 258 52 286 28
2015.Fall 37 263 52 319 56
2014.Fall 37 223 52 304 81
2013.Fall 37 270 52 299 29
2012.Fall 37 327 52 365 38


In [16]:
weeks = range(1, 53)
for w in weeks:
    f = f"{w:02d}"
    yet_to_deposit = (
        summ.loc[("2018.Fall", "Deposited")]["52"]
        - summ.loc[("2018.Fall", "Deposited")][f]
    )
    print(w, yet_to_deposit)


1 238
2 239
3 239
4 239
5 239
6 239
7 239
8 239
9 239
10 239
11 239
12 239
13 239
14 238
15 238
16 238
17 238
18 238
19 237
20 234
21 222
22 210
23 195
24 186
25 184
26 164
27 154
28 147
29 141
30 135
31 133
32 121
33 112
34 103
35 81
36 63
37 59
38 52
39 49
40 46
41 42
42 40
43 27
44 22
45 18
46 12
47 9
48 9
49 6
50 4
51 2
52 0


In [17]:
summ.describe()


,00,01,02,03,04,05,06,07,08,09,...,44,45,46,47,48,49,50,51,52,53
count,48.0,48.000000,48.000000,48.000000,48.000000,48.000000,48.000000,48.000000,48.000000,48.000000,...,48.000000,48.000000,48.000000,48.000000,48.000000,48.000000,48.000000,48.000000,48.000000,48.000000
mean,0.0,16.208333,16.916667,18.854167,21.354167,23.666667,26.479167,30.875000,35.666667,42.875000,...,276.770833,274.562500,273.020833,271.916667,266.416667,256.479167,246.708333,241.375000,238.895833,238.895833
std,0.0,22.227028,22.420387,25.267917,28.921382,31.951648,36.116236,42.604377,50.261585,59.944167,...,312.330924,309.647425,308.449692,307.197825,301.812809,288.847452,279.583502,276.912431,275.162562,275.162562
min,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,0.0,2.000000,2.750000,3.000000,3.000000,3.000000,3.000000,4.750000,5.750000,5.750000,...,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000
50%,0.0,8.000000,8.000000,8.000000,9.500000,10.500000,10.500000,14.000000,16.000000,16.500000,...,112.000000,112.000000,111.500000,111.500000,111.500000,111.500000,111.500000,111.500000,111.500000,111.500000
75%,0.0,18.250000,20.500000,24.000000,25.250000,28.500000,36.250000,37.750000,41.000000,58.250000,...,440.250000,429.000000,419.000000,404.250000,378.250000,363.000000,349.500000,338.000000,326.750000,326.750000
max,0.0,90.000000,89.000000,103.000000,124.000000,132.000000,147.000000,175.000000,211.000000,246.000000,...,1024.000000,992.000000,984.000000,974.000000,935.000000,934.000000,922.000000,927.000000,921.000000,921.000000


In [18]:
summ.index


MultiIndex(levels=[['2012.Fall', '2012.Spring', '2013.Fall', '2013.Spring', '2014.Fall', '2014.Spring', '2015.Fall', '2015.Spring', '2016.Fall', '2016.Spring', '2017.Fall', '2017.Spring', '2018.Fall', '2018.Spring', '2019.Fall', '2019.Spring'], ['Accepted', 'Applied', 'Deposited']],
           codes=[[0, 0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4, 5, 5, 5, 6, 6, 6, 7, 7, 7, 8, 8, 8, 9, 9, 9, 10, 10, 10, 11, 11, 11, 12, 12, 12, 13, 13, 13, 14, 14, 14, 15, 15, 15], [0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2]],
           names=['year_term', 'stage'])

In [19]:
# Goal
GOAL = 260
summ.loc[("2019.Fall", "Deposited")]["52"] = GOAL


In [20]:
df1 = summ.copy()
weeks = range(1, 53)
for w in weeks:
    f = f"{w:02d}"
    df1[f] = summ.loc[:]["52"] - summ.loc[:][f]

print(df1.shape)


(48, 54)


In [21]:
df1.head()


00   01   02   03   04   05   06   07   08   09  ...  \
year_term   stage                                                       ...   
2012.Fall   Accepted    0  275  275  275  275  275  275  275  274  274  ...   
            Applied     0  422  422  422  421  418  417  416  412  409  ...   
            Deposited   0  362  362  362  362  362  362  362  361  361  ...   
2012.Spring Accepted    0    1    1    1    1    1    1    1    1    1  ...   
            Applied     0    2    2    2    2    1    1    1    1    1  ...   

                        44   45   46   47  48  49  50  51  52   53  
year_term   stage                                                   
2012.Fall   Accepted  -140 -137 -120 -119 -83 -84 -67 -14   0  276  
            Applied   -162 -160 -139 -134 -86 -87 -70 -13   0  423  
            Deposited   -3   -3   -4   -4  -7  -7  -8   0   0  365  
2012.Spring Accepted     0    0    0    0   0   0   0   0   0    1  
            Applied      0    0    0    0   0   0   0   0   0    2  

[5 rows x 54 columns]

In [22]:
df1_t = df1.transpose()


In [23]:
df1_t.head()


year_term 2012.Fall                   2012.Spring                   2013.Fall  \
stage      Accepted Applied Deposited    Accepted Applied Deposited  Accepted   
00                0       0         0           0       0         0         0   
01              275     422       362           1       2        39       279   
02              275     422       362           1       2        39       277   
03              275     422       362           1       2        39       274   
04              275     421       362           1       2        39       273   

year_term                   2013.Spring  ... 2018.Fall 2018.Spring          \
stage     Applied Deposited    Accepted  ... Deposited    Accepted Applied   
00              0         0           0  ...         0           0       0   
01            230       290          21  ...       238          21      23   
02            240       290          21  ...       239          21      23   
03            236       289          21  ...       239          21      22   
04            220       286          16  ...       239          21      21   

year_term           2019.Fall                   2019.Spring                    
stage     Deposited  Accepted Applied Deposited    Accepted Applied Deposited  
00                0         0       0         0           0       0         0  
01               18       883     869       259          21      16        13  
02               18       883     859       259          19      13        13  
03               18       883     851       259          18      11        12  
04               18       883     842       259          18      10        12  

[5 rows x 48 columns]

In [24]:
# from bokeh.layouts import widgetbox, row
# from bokeh.models.widgets import MultiSelect, RadioGroup, Select
# from bokeh.plotting import figure, curdoc
from bokeh.plotting import figure, output_file, show

# from bokeh.palettes import Blues9
from bokeh.palettes import Set2_8
from bokeh.io import output_notebook


In [25]:
output_notebook()
output_file("DepositsToEndOFYear.html")


Loading BokehJS ...

In [26]:
TOOLS = "pan,wheel_zoom,box_zoom,save,reset"


In [27]:
stage = "Deposited"
term = "2019.Fall"

all_terms = sorted(list(df["year_term"].dropna().unique()))
all_terms = [l for l in all_terms if "Fall" in l]
all_terms.remove(term)
# all_terms.remove('2013.Fall')
all_terms.remove("2012.Fall")
all_terms.reverse()
y_max = df1_t[(term, stage)].max()
y_min = df1_t[(term, stage)].min()
for t in all_terms:
    ymx = df1_t[(t, stage)].max()
    ymn = df1_t[(t, stage)].min()
    if ymx > y_max:
        y_max = ymx
    if ymn < y_min:
        y_min = ymn

print(term, stage, y_min, y_max)


2019.Fall Deposited -15 319


In [28]:
print(all_terms)


['2018.Fall', '2017.Fall', '2016.Fall', '2015.Fall', '2014.Fall', '2013.Fall']


In [29]:
title = f"'{stage}' Gained to End of Year,  Goal={GOAL}  (Week {adm_week_number:d})"


In [30]:
p = figure(
    plot_width=800,
    plot_height=600,
    title=title,
    x_axis_label="Admissions Week Number (year starts Sept 1)",
    y_axis_label=stage,
    tools=TOOLS,
    x_range=(0, 54),
    y_range=(y_min * 1.05, y_max * 1.05),
)

c = 0
for t in all_terms:
    # p.line(df1_t.index, df1_t[(t, stage)], color=Blues9[c], legend=t)
    p.line(df1_t.index, df1_t[(t, stage)], color=Set2_8[c], legend=t)
    if c <= 7:
        c += 1

p.line(df1_t.index, df1_t[(term, stage)], color="red", line_width=2, legend=term)

# week_number line
p.line(
    (adm_week_number, adm_week_number),
    (-1000, 5000),
    color="green",
    line_width=0.8,
    line_dash="dashed",
    legend=f"Week {adm_week_number:d}",
    alpha=0.8,
)

p.legend.location = "top_right"

show(p)


In [31]:
# get list of 2019.Fall applicants
dep = (
    df.loc[
        (df["year_term"] == "2019.Fall") & (df["stage"] == "Deposited") & (df["53"] > 0)
    ]
    .sort_values(["PEOPLE_CODE_ID"])
    .drop_duplicates(["PEOPLE_CODE_ID"])
)
print(dep.shape)


(176, 58)


In [32]:
dep.head()


,year_term,stage,PEOPLE_CODE_ID,00,01,02,03,04,05,06,...,45,46,47,48,49,50,51,52,53,curriculum
36964,2019.Fall,Deposited,P000035178,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,ECOR
36966,2019.Fall,Deposited,P000047813,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,NRCM
36967,2019.Fall,Deposited,P000050419,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,NRCM
36968,2019.Fall,Deposited,P000052458,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,PACM
36969,2019.Fall,Deposited,P000053667,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,FEFM


In [33]:
dep.to_csv("2019-Fall-Deposited.csv")


In [34]:
df.loc[(df["PEOPLE_CODE_ID"] == "P000053667")]


,year_term,stage,PEOPLE_CODE_ID,00,01,02,03,04,05,06,...,45,46,47,48,49,50,51,52,53,curriculum
9968,2017.Fall,Applied,P000053667,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,NaN
11697,2019.Fall,Applied,P000053667,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,FEFM
22604,2017.Fall,Accepted,P000053667,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,NaN
24333,2019.Fall,Accepted,P000053667,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,FEFM
35240,2017.Fall,Deposited,P000053667,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
36969,2019.Fall,Deposited,P000053667,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,FEFM


In [30]:
# melt
import numpy as np

a = [
    summ.loc[("2017.Fall", "Deposited")]["53"]
    - summ.loc[("2017.Fall", "Deposited")][str(week)],
    summ.loc[("2016.Fall", "Deposited")]["53"]
    - summ.loc[("2016.Fall", "Deposited")][str(week)],
    summ.loc[("2015.Fall", "Deposited")]["53"]
    - summ.loc[("2015.Fall", "Deposited")][str(week)],
    summ.loc[("2014.Fall", "Deposited")]["53"]
    - summ.loc[("2014.Fall", "Deposited")][str(week)],
    summ.loc[("2013.Fall", "Deposited")]["53"]
    - summ.loc[("2013.Fall", "Deposited")][str(week)],
    summ.loc[("2012.Fall", "Deposited")]["53"]
    - summ.loc[("2012.Fall", "Deposited")][str(week)],
]
print(a)
print(sum(a))
print(sum(a) / 6)
print(np.mean(a))


[2, -4, -6, 0, -2, 0]
-10
-1.6666666666666667
-1.6666666666666667


In [40]:
for st in stages:
    df = summ_t.loc[:, (terms, st)]
    

In [41]:
df

year_term,2012.Fall,2013.Fall,2014.Fall,2015.Fall,2016.Fall,2017.Fall,2018.Fall,2019.Fall
stage,Deposited,Deposited,Deposited,Deposited,Deposited,Deposited,Deposited,Deposited
00,0,0,0,0,0,0,0,0
01,3,9,4,2,1,10,7,1
02,3,9,4,2,1,10,6,1
03,3,10,4,2,1,11,6,1
04,3,13,4,2,1,11,6,1
05,3,13,4,2,1,11,6,1
06,3,13,4,2,1,11,6,1
07,3,14,4,2,1,11,6,1
08,4,16,4,4,1,11,6,1


In [56]:
summ = df.groupby(["year_term", "stage"]).sum()
summ_t = summ.transpose()
summ_t = summ_t.reset_index().rename(columns={"index": "week_num"})
summ_t["wn"] = summ_t.loc[:, ["week_num"]].astype(int)
summ_t.head()

year_term week_num 2012.Fall                   2012.Spring                    \
stage               Accepted Applied Deposited    Accepted Applied Deposited   
0               00         0       0         0           0       0         0   
1               01         1       1         3           0       0         0   
2               02         1       1         3           0       0         0   
3               03         1       1         3           0       0         0   
4               04         1       2         3           0       0         0   

year_term 2013.Fall                    ... 2018.Spring                    \
stage      Accepted Applied Deposited  ...    Accepted Applied Deposited   
0                 0       0         0  ...           0       0         0   
1                18      83         9  ...           2       4         3   
2                20      73         9  ...           2       4         3   
3                23      77        10  ...           2       5         3   
4                24      93        13  ...           2       6         3   

year_term 2019.Fall                   2019.Spring                   wn  
stage      Accepted Applied Deposited    Accepted Applied Deposited     
0                 0       0         0           0       0         0  0  
1                 2      19         1           2       8         2  1  
2                 2      29         1           4      11         2  2  
3                 2      37         1           5      13         3  3  
4                 2      46         1           5      14         3  4  

[5 rows x 50 columns]

In [57]:
week_end = (
    lambda r: adm_week_end_date(int(r["wn"]), acad_yr)
)
summ_t["Week_End"] = summ_t.apply(week_end, axis=1)


In [58]:
summ_t = summ_t.drop(columns=["wn", "week_num"], axis=1).set_index("Week_End")

In [59]:
summ_t.head()

year_term  2012.Fall                   2012.Spring                    \
stage       Accepted Applied Deposited    Accepted Applied Deposited   
Week_End                                                               
2018-08-26         0       0         0           0       0         0   
2018-09-02         1       1         3           0       0         0   
2018-09-09         1       1         3           0       0         0   
2018-09-16         1       1         3           0       0         0   
2018-09-23         1       2         3           0       0         0   

year_term  2013.Fall                   2013.Spring  ... 2018.Fall 2018.Spring  \
stage       Accepted Applied Deposited    Accepted  ... Deposited    Accepted   
Week_End                                            ...                         
2018-08-26         0       0         0           0  ...         0           0   
2018-09-02        18      83         9           8  ...         7           2   
2018-09-09        20      73         9           8  ...         6           2   
2018-09-16        23      77        10           8  ...         6           2   
2018-09-23        24      93        13          13  ...         6           2   

year_term                    2019.Fall                   2019.Spring          \
stage      Applied Deposited  Accepted Applied Deposited    Accepted Applied   
Week_End                                                                       
2018-08-26       0         0         0       0         0           0       0   
2018-09-02       4         3         2      19         1           2       8   
2018-09-09       4         3         2      29         1           4      11   
2018-09-16       5         3         2      37         1           5      13   
2018-09-23       6         3         2      46         1           5      14   

year_term             
stage      Deposited  
Week_End              
2018-08-26         0  
2018-09-02         2  
2018-09-09         2  
2018-09-16         3  
2018-09-23         3  

[5 rows x 48 columns]

In [62]:
print(today_str)
writer = pd.ExcelWriter(
    f"{term}_HistoricAdmissionsData_{b_yr:d}-{e_yr:d}_{today_str}.xlsx"
)
for st in stages:
    df_sumt = summ_t.loc[:, (terms, st)]
    #df_sumt.T.reset_index().T.to_excel(writer, st)
    df_sumt.to_excel(writer, st)

writer.save()


20190508
